In [10]:
# Importing libraries
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler 
import warnings
warnings.filterwarnings('ignore')
import scipy
from sklearn.cluster import KMeans

In [2]:
# Importing files
precip = pd.read_excel(r"C:\Users\joaoa\Desktop\Doutoramento\Precipitacao_diaria_2\ERA5_Rainfall_PT_mm_Oct1981_Sep2023.xlsx")
local = pd.read_csv(r"C:\Users\joaoa\Desktop\Doutoramento\Professora\Nova pasta\Todos os Pontos (temperatura)\local.cvs")

# Checking nulls
print("Nulls:", precip.isnull().sum().sum())

# Creating column "year"
precip.insert(0, "year", 0)
precip.insert(1, "day", 0)
for row in range(0,len(precip)):
    precip.loc[row, "year"] = 1981 + row // 365
    precip.loc[row, "day"] = (row + 1) - 365*(row // 365)
precip = precip.drop(["Index"], axis = 1)
precip

Nulls: 0


,year,day,P0000,P0001,P0002,P0003,P0004,P0005,P0006,P0007,...,P1047,P1048,P1055,P1056,P1057,P1058,P1059,P1062,P1063,P1064
0,1981,1,18.59,29.60,28.42,25.24,21.88,19.20,17.59,28.02,...,0.44,0.51,0.25,0.24,0.25,0.26,0.27,0.15,0.15,0.14
1,1981,2,3.65,9.55,8.22,6.58,4.94,3.69,2.98,10.32,...,0.02,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1981,3,7.53,15.72,14.61,12.94,11.11,9.19,7.73,15.97,...,2.12,2.66,1.65,1.61,1.32,0.93,0.59,1.18,0.99,0.83
3,1981,4,51.39,47.14,49.84,52.24,54.10,53.28,50.52,45.37,...,0.29,0.47,0.53,0.49,0.43,0.35,0.28,0.25,0.24,0.22
4,1981,5,51.86,30.62,38.98,48.55,57.39,62.94,66.47,28.54,...,9.71,10.47,13.67,13.27,12.35,11.22,10.13,10.34,9.65,9.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15325,2022,361,0.05,0.03,0.02,0.02,0.02,0.02,0.02,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15326,2022,362,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15327,2022,363,0.06,0.50,0.38,0.24,0.10,0.03,0.01,0.47,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15328,2022,364,0.06,0.01,0.02,0.03,0.05,0.05,0.04,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [35]:
# Creating a dictionary with all dataframes
df_dict = {}

for days in range(1,11):
    # Size of the group
    d = days

    # Rolling sum
    df_roll = precip
    df_roll = df_roll.rolling(d).sum()
    df_roll[["year","day"]] = precip[["year","day"]]
    df_roll = df_roll.loc[df_roll["day"] >= d, :]
    df_roll = df_roll.drop(["day"], axis = 1)

    # Grouping by "year" computing its maximum value
    max_precip = df_roll.groupby("year").max()
    max_precip.index.name = None

    # Standardizing dataframe
    scaler = StandardScaler() 
    stand_precip = scaler.fit_transform(max_precip) 
    stand_precip = pd.DataFrame(stand_precip, columns = max_precip.columns)
    stand_precip = stand_precip*np.sqrt((len(stand_precip)-1)/len(stand_precip))

    # Transposing dataframe
    transp = stand_precip.transpose()

    # Clustering (k-means)
    K = range(2, 21)
    inertia = []
    X = transp
    X.columns = X.columns.astype(str)

    for n in K:
        kmeans = KMeans(n_clusters = n, random_state = 42)
        kmeans.fit(X)
        inertia.append(kmeans.inertia_)
        kmeans.labels_ += 1
        transp[str(n) + "_clusters|" + str(d) + "_days"] = kmeans.labels_
    
    df_dict[str(d) + "_days"] = transp

In [36]:
df_dict["10_days"]

,0,1,2,3,4,5,6,7,8,9,...,11_clusters|10_days,12_clusters|10_days,13_clusters|10_days,14_clusters|10_days,15_clusters|10_days,16_clusters|10_days,17_clusters|10_days,18_clusters|10_days,19_clusters|10_days,20_clusters|10_days
P0000,-0.003467,-0.478665,1.044018,1.380931,-0.094948,-1.108975,0.112383,-0.433602,2.135600,-0.379062,...,5,4,2,2,4,6,6,8,5,3
P0001,0.090087,-0.432662,0.883191,0.764274,-0.059247,-1.378983,0.076335,-0.740714,1.880638,-0.198711,...,5,4,2,2,4,6,6,8,5,3
P0002,0.064071,-0.497605,0.863455,0.864428,-0.044405,-1.356335,0.067152,-0.635754,2.009670,-0.175258,...,5,4,2,2,4,6,6,8,5,3
P0003,0.061740,-0.495858,0.828168,0.957838,-0.100720,-1.317102,0.028453,-0.577005,2.198067,-0.159676,...,5,4,2,2,4,6,6,8,5,3
P0004,0.064430,-0.456163,0.789360,1.053217,-0.139059,-1.251805,-0.009165,-0.527553,2.352664,-0.150251,...,5,4,2,2,4,6,6,8,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P1058,-0.562733,-1.125855,1.740566,-0.536758,-1.021952,-0.624354,1.240772,1.831765,3.102486,-0.890178,...,7,7,9,9,12,16,1,15,14,5
P1059,-0.540999,-1.123194,1.832231,-0.550876,-1.061271,-0.626286,1.138155,1.784553,3.087227,-0.917859,...,7,7,9,9,12,16,1,15,14,5
P1062,-0.345651,-1.042452,1.720370,-0.428590,-0.957020,-0.638788,1.657721,1.548617,3.058087,-0.852901,...,7,7,9,9,12,16,1,15,14,5
P1063,-0.316978,-1.033635,1.792775,-0.471628,-0.968857,-0.622541,1.566406,1.544517,3.033000,-0.866528,...,7,7,9,9,12,16,1,15,14,5


In [43]:
# Creating dataframes based on number of clusters
df_index = pd.DataFrame(index = transp.index)

for n in K:
    df_aux = df_index
    for d in range(1,11):
        df_aux = pd.concat([df_aux, df_dict[str(d) + "_days"].iloc[:, 40+n]], axis = 1)
        
    df_dict[str(n) + "_clusters"] = df_aux

In [44]:
df_dict["3_clusters"]

,3_clusters|1_days,3_clusters|2_days,3_clusters|3_days,3_clusters|4_days,3_clusters|5_days,3_clusters|6_days,3_clusters|7_days,3_clusters|8_days,3_clusters|9_days,3_clusters|10_days
P0000,3,3,3,3,3,2,1,3,1,3
P0001,3,3,3,3,3,2,1,3,1,3
P0002,3,3,3,3,3,2,1,3,1,3
P0003,3,3,3,3,3,2,1,3,1,3
P0004,3,3,3,3,3,2,1,3,1,3
...,...,...,...,...,...,...,...,...,...,...
P1058,2,1,1,2,2,3,3,2,3,2
P1059,2,1,1,2,2,3,3,2,3,2
P1062,2,1,1,2,2,3,3,2,3,2
P1063,2,1,1,2,2,3,3,2,3,2
